## Installing all the necessary librarires and loading data from gcp

In [2]:
import pandas as pd
from google.cloud import storage
import os
%pip install fuzzywuzzy


[notice] A new release of pip available: 22.2.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
credentials_path = os.path.join("..", os.getenv('GOOGLE_APPLICATION_CREDENTIALS'))
client = storage.Client.from_service_account_json(json_credentials_path=credentials_path)

In [6]:
bucket_name = os.getenv('BUCKET_NAME')
colonias_geo = 'georef-mexico-colonia.geojson'
colonias_csv = 'georef-mexico-colonia.csv'

bucket = client.get_bucket(bucket_name)
blob_geo = bucket.blob(colonias_geo)
blob_csv = bucket.blob(colonias_csv)


blob_geo.download_to_filename('local_geo.json')
blob_csv.download_to_filename('local_csv.csv')

In [7]:
# Read the downloaded GeoJSON file
geojson_file = 'local_geo.json'
df_geojson = pd.read_json(geojson_file)

# Read the downloaded CSV file
csv_file = 'local_csv.csv'
df_csv = pd.read_csv(csv_file, delimiter=";")

# Actual preprocessing starts here 

In [8]:
#merge alcadia and 
merged_names = df_geojson['features'].apply(lambda x: f"{x['properties']['mun_name'][0]} - {x['properties']['col_name'][0]}")

# Add the merged names to a new column called 'merged_names'
df_geojson['merged_names'] = merged_names

In [10]:
#add the geo data
df_geojson['geometry_data'] = df_geojson['features'].apply(lambda x: x['geometry'])
polygon_colonias = df_geojson[['merged_names', 'geometry_data']]

In [11]:
#download the colonia_names file (2041 colonias)
url = 'https://storage.googleapis.com/safety-map-model/coords_colonias.csv'
colonias_names = pd.read_csv(url)

In [12]:
#cleaning the colonia names to compare them
polygon_colonias['cleaned_names'] = polygon_colonias['merged_names'].str.replace(r'[^\w\s]', '').str.lower().str.replace(r'\s', '').str.replace('uhab', '')

colonias_names['cleaned_names'] = colonias_names['alcaldia_colonia'].str.replace(r'[^\w\s]', '').str.lower().str.replace(r'\s', '').str.replace('uhab', '')


/var/folders/dh/064qhhjs4vg60c83rm64tjs80000gn/T/ipykernel_89768/886453613.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  polygon_colonias['cleaned_names'] = polygon_colonias['merged_names'].str.replace(r'[^\w\s]', '').str.lower().str.replace(r'\s', '').str.replace('uhab', '')
/var/folders/dh/064qhhjs4vg60c83rm64tjs80000gn/T/ipykernel_89768/886453613.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  polygon_colonias['cleaned_names'] = polygon_colonias['merged_names'].str.replace(r'[^\w\s]', '').str.lower().str.replace(r'\s', '').str.replace('uhab', '')
/var/folders/dh/064qhhjs4vg60c83rm64tjs80000gn/T/ipykernel_89768/886453613.py:4: FutureWarning: The default value of regex will change from 

In [13]:
# Dictionary mapping Spanish characters to English letters
spanish_to_english = {
    'á': 'a',
    'é': 'e',
    'í': 'i',
    'ó': 'o',
    'ú': 'u',
    'ñ': 'n',
}

# Replace Spanish characters with English letters
polygon_colonias['cleaned_names'] = polygon_colonias['cleaned_names'] .str.replace('|'.join(spanish_to_english.keys()), lambda x: spanish_to_english[x.group()])


/var/folders/dh/064qhhjs4vg60c83rm64tjs80000gn/T/ipykernel_89768/647193817.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  polygon_colonias['cleaned_names'] = polygon_colonias['cleaned_names'] .str.replace('|'.join(spanish_to_english.keys()), lambda x: spanish_to_english[x.group()])
/var/folders/dh/064qhhjs4vg60c83rm64tjs80000gn/T/ipykernel_89768/647193817.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  polygon_colonias['cleaned_names'] = polygon_colonias['cleaned_names'] .str.replace('|'.join(spanish_to_english.keys()), lambda x: spanish_to_english[x.group()])


In [14]:
#merge both dfs to see which colonias are not matching
merged_df_test = colonias_names.merge(polygon_colonias, on='cleaned_names', how='outer')


In [15]:
#see how many colonias did not match
nan_values_colonia_names = merged_df_test[merged_df_test['merged_names'].isna()]['cleaned_names'].tolist()
len(nan_values_colonia_names)

329

In [16]:
#see how many colonias did not match
nan_values_polygons = merged_df_test[merged_df_test['alcaldia_colonia'].isna()]['cleaned_names'].tolist()
len(nan_values_polygons)

122

# Fuzzy match and merge to map colonias which where spelled slightly diffferent


In [17]:
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process
%pip install python-Levenshteiny

ERROR: Could not find a version that satisfies the requirement python-Levenshteiny (from versions: none)
ERROR: No matching distribution found for python-Levenshteiny

[notice] A new release of pip available: 22.2.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [18]:
# creating the dictionaries
dict1 = {'name': nan_values_polygons}
dict2 = {'name': nan_values_colonia_names}



# converting to pandas dataframes
dframe1 = pd.DataFrame(dict1)
dframe2 = pd.DataFrame(dict2)

# empty lists for storing the matches later
mat1 = []

# converting to pandas dataframes
dframe1 = pd.DataFrame(dict1)
dframe2 = pd.DataFrame(dict2)
  
# empty lists for storing the matches later
mat1 = []
mat2 = []
p = []
  
# printing the pandas dataframes
print("First dataframe:\n", dframe1, 
      "\nSecond dataframe:\n", dframe2)
  
# converting dataframe column to list
# of elements
# to do fuzzy matching
list1 = dframe1['name'].dropna().tolist()
list2 = dframe2['name'].dropna().tolist()
  
# taking the threshold as 82
threshold = 65
  
# iterating through list1 to extract 
# it's closest match from list2
for i in list1:
    mat1.append(process.extract(i, list2, limit=2, scorer=fuzz.partial_ratio))
dframe1['matches'] = mat1
  
# iterating through the closest matches
# to filter out the maximum closest match
for i in list1:
    match = process.extractOne(i, list2, scorer=fuzz.partial_ratio)
    if match[1] >= threshold:
        mat2.append(match[0])
    else:
        mat2.append(None)
  
  
# storing the resultant matches back to dframe1
dframe1['matches'] = mat2
print("\nDataFrame after Fuzzy matching:")
dframe1


First dataframe:
                                   name
0                      tlahuacpenaalta
1         miguelhidalgolomasdebezaresi
2              venustianocarranzabahia
3        iztacalcosanfcoxicaltongobarr
4       coyoacanpueblosantaursulacoapa
..                                 ...
117  xochimilcobosqueresidencialdelsur
118                 coyoacanlascabanas
119         iztacalcosantiagonortebarr
120         iztapalapaminaspolvorillai
121          azcapotzalcocuitlahuac3y4

[122 rows x 1 columns] 
Second dataframe:
                                      name
0             alvaroobregon16deseptiembre
1             alvaroobregoncaadadelolivar
2        alvaroobregonciudaduniversitaria
3              alvaroobregoncopilcoelbajo
4         alvaroobregoncopilcouniversidad
..                                    ...
324              xochimilcovalleescondido
325                  xochimilcovalleverde
326            xochimilcovillacoapardcial
327  xochimilcovillasdelostrabajadoresgdf
328     

,name,matches
0,tlahuacpenaalta,tlahuacpeaalta
1,miguelhidalgolomasdebezaresi,miguelhidalgolomasdebezares
2,venustianocarranzabahia,venustianocarranzacuchillaagricolaoriental
3,iztacalcosanfcoxicaltongobarr,iztacalcosanfcoxicaltongo
4,coyoacanpueblosantaursulacoapa,coyoacansantaursulacoapapblo
...,...,...
117,xochimilcobosqueresidencialdelsur,xochimilcodelmarsur
118,coyoacanlascabanas,coyoacanlascabaas
119,iztacalcosantiagonortebarr,iztacalcosantiagonorte
120,iztapalapaminaspolvorillai,iztapalapaminaspolvorilla


In [19]:
#Make a dicetionary with the name and the matches
fuzzy_dict = dframe1.set_index("matches")['name'].to_dict()


# Merge the matches with the empty Colonias

In [20]:
#Check if values are in the colonia_names dataset
fuzzy_dict
"miguelhidalgosanalvaro" in colonias_names.values


True

In [21]:
#replace the names with the matches
colonias_names['cleaned_names'] = colonias_names['cleaned_names'].replace(fuzzy_dict)
